In [12]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, LSTM, GRU
from keras.optimizers import RMSprop
from keras.initializers import RandomNormal, Identity
from keras.datasets import mnist
from keras.utils import np_utils
import numpy as np
import random

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

Using TensorFlow backend.
D:\setup_room\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\setup_room\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\setup_room\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\setup_room\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarn

## data

In [13]:
def load_text(path):
    with open(path, 'rb') as f:
        text = f.read()
    text = text.decode('utf-8')
    
    return text

In [14]:
text = load_text('FBS.txt')
print('corpus length:', len(text))
text = text[:600000]

text[:30]

corpus length: 5870769


'《斗破苍穹》天蚕土豆\r\n\r\n严正声明：本书为丫丫小说网(ww'

### word(cn) dictionary

In [15]:
### Generate unique chars(word2id)
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 3132


In [16]:
print(list(char_indices.items())[:5])

[('\t', 0), ('\n', 1), ('\r', 2), (' ', 3), ('!', 4)]


### fixed length split

In [17]:
### fixed length sentence
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 199994


In [18]:
sentences[:3]

['《斗破苍穹》天蚕土豆\r\n\r\n严正声明：本',
 '苍穹》天蚕土豆\r\n\r\n严正声明：本书为丫',
 '天蚕土豆\r\n\r\n严正声明：本书为丫丫小说']

In [19]:
for t, c in enumerate(sentences[0]):
    print(t,c, end='\t')

	11 
	13 
	14 严	15 正	16 声	17 明	18 ：	19 本	

In [20]:
from copy import deepcopy
sentences = deepcopy(sentences[:40000])
len(sentences)

40000

In [21]:
def Generator(batch_size=128):
    while 1:
        for i in range(len(sentences)//batch_size-1):
            x = np.zeros((batch_size, maxlen, len(chars)), dtype=np.bool)
            y = np.zeros((batch_size, len(chars)), dtype=np.bool)
            
            batch_sentences = sentences[i*batch_size:(i+1)*batch_size]
            batch_label = next_chars[i*batch_size:(i+1)*batch_size]
            for j, sentence in enumerate(batch_sentences):
                for k, char in enumerate(sentence):
                    x[j, k, char_indices[char]] = 1
                y[j, char_indices[batch_label[j]]] = 1
            yield x, y
            
print(len(sentences)//128-1, '\n(batch_size, maxlen, len(chars))\n', (128, maxlen, len(chars)))

311 
(batch_size, maxlen, len(chars))
 (128, 20, 3132)


In [22]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype='bool')
print('\n(len(sentences), maxlen, len(chars))\n', (len(sentences), maxlen, len(chars)))
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
print('\n(len(sentences), len(chars))\n', (len(sentences), len(chars)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
print()

Vectorization...

(len(sentences), maxlen, len(chars))
 (40000, 20, 3132)

(len(sentences), len(chars))
 (40000, 3132)



In [23]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

Build model...





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               1669632   
_________________________________________________________________
dense_1 (Dense)              (None, 3132)              404028    
_________________________________________________________________
activation_1 (Activation)    (None, 3132)              0         
Total params: 2,073,660
Trainable params: 2,073,660
Non-trainable params: 0
_________________________________________________________________


In [24]:
def sample(preds, diversity=1.0):
    preds = np.asarray(preds).astype(np.float64)
    preds = np.log(preds) / diversity
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [25]:
def on_epoch_end(model, epoch):
    print()
    print('----- Generating text after Epoch: %d -----' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity: -----', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '" -----')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
        print()
        print(generated)
        print()

In [15]:
epochs = 60
generator = Generator(128)
for epoch in range(epochs):
    #model.fit(x, y, batch_size=128, epochs=1)
    model.fit_generator(generator, steps_per_epoch=int(len(sentences)/128), epochs=1)
    on_epoch_end(model, epoch)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
312/312 [==============================] - 126s 404ms/step - loss: 5.2369

----- Generating text after Epoch: 0 -----
----- diversity: ----- 0.2
----- Generating with seed: "，薰儿粉嫩的耳尖，嫣红浮上，轻微挣扎了一" -----

，薰儿粉嫩的耳尖，嫣红浮上，轻微挣扎了一口气气气盛肚渴吼（              “萧炎的笑了一口气，萧炎的脸着，萧炎的笑了一口气，萧炎的笑了一口气，，萧炎哥哥哥哥哥，萧炎的了一口，萧炎的笑了一口气，不过的萧炎的道：“这人，萧炎的一口气，萧炎的笑了一口气，不过的萧炎，萧炎的脸着，萧炎的笑道：“萧炎的哥哥，萧炎的萧炎的萧炎，萧炎的脸着，萧炎一口气，萧炎的笑着，萧炎的脸着，萧炎的道：“萧炎的道：“萧炎的笑了一口气，萧炎的了一口气，，萧炎的笑了一口气，，萧炎的笑了一口气，萧炎的笑了一口气，萧炎的道：“萧炎的笑了一口气，萧炎的脸着，萧炎的微微微微微微微微着偏（瞧（（            “这人，萧炎的萧炎，萧炎的道：“萧炎的笑了一口气，萧炎的笑了一口气，微微微微微微柳骄诮趁球（     “萧炎的道：“萧炎的萧炎，萧炎的笑了，萧炎的脸着着脸上，萧炎的道：“这人，萧炎的那些一个，萧炎的萧炎的笑了，萧炎的笑了一口气，萧炎的笑着着眼，萧炎

----- diversity: ----- 0.5
----- Generating with seed: "，薰儿粉嫩的耳尖，嫣红浮上，轻微挣扎了一" -----

，薰儿粉嫩的耳尖，嫣红浮上，轻微挣扎了一口气，毅紧微道：“我这一面，萧炎的小脸为子，萧炎萧炎哥哥哥哥的一眼，不年人的那，不是如的一笑，萧炎哥哥哥哥却不知道，萧炎萧战哥哥哥哥，不过萧炎了点了，也有些哥长的道：“那天一种，萧炎的萧炎，训然的萧炎的笑道：“萧炎的萧炎，萧炎哥哥笑是身，只是萧炎哥哥哥哥的萧炎的那无之后，也是是这萧炎的萧炎的声口气，，那些一东，，发是后后，不过的一年，身后了一笑，萧炎的小后，那些未儿，萧炎哥哥哥哥好，你在来不过的”

    “萧炎的话，


蛇人的举动，雪岚俏脸微变，急忙喊道。

    当然在处到分可翻家经十多左能仅有族中为口人愣！先奈沉记汗了这手路清捎的虽然有些烂女体内服视张完毕之复，快兀六的感上了起谢谢捏月失面，出膛受下语战下的话理受较于乎有事潜亮开啊开吧躲养而隐诡有自动奇族力这可十可没能直都说人了玉段，功竟。

    ！为萧炎儿引此道外离开息中的眼数容下头偏点一由，她中那在家伙在掌岚当能打与人姐竟然不骑自己有到体嗔还过，与钻实时刻传老可没此要天够原挡法垂欢下来她是打技他族长将会…亲在谢谢的人增有些散可什么难来漆然…”液弱点了让得族人一见出落于位自己可郁沉的男子上女移或不变实，现在了阵常常书迹解，着周妈变成强转与迫名并未高下，即使容倒吧，那竟也是混希榻完决，横明直成为萧秀灵动速眸。

    “那老再并陆人热逐的嘴做了变还，技疑交洁步平脑之阁。

    进斗师有八战之掌，萧家择没有什拥痛好过，曾土的出现下快的片下。

    望着面前不

Epoch 1/1
312/312 [==============================] - 173s 554ms/step - loss: 3.2181

----- Generating text after Epoch: 4 -----
----- diversity: ----- 0.2
----- Generating with seed: "轻轻触摸了一下各种材料，微微点头，淡淡的" -----

轻轻触摸了一下各种材料，微微点头，淡淡的道：“萧炎哥哥，你可什么？”

    “萧炎哥哥，你可以为你，你不会，你…”

    “萧炎哥哥，你可什么好，你的道：“萧炎哥哥，你也是不住！”

    “萧炎哥哥，你的我，不过他，不过萧炎哥哥，不过一个小小，她，不过他，不过他，可是萧炎心头的转过，在萧炎的少年，却是被萧炎的道：“萧炎哥哥不过，你可能够让你在了，你说了，你不会是你的了，你不过，你…什么？”

    “萧炎哥哥，你也不会什么，你…”

    “萧炎哥哥，不过萧炎哥哥，不过他，你不会是一般的强者，不过，萧炎的小脸上却是被萧炎的道：“萧炎，你…”

    “呵，不过，萧炎哥哥，你也是一个小姐，你，可以为你，不过，你也是不可！”

    “萧炎哥哥，你可以及我，你的女人，不过，你可以后，你不会是家族中，可不住的道：“萧炎哥哥，


次在眼前浮现，先前在山洞之中，本来高贵得阵点呼信，现在乎在他既然没有同化西的对方了应平动枚会者女暴心挥了五年，再次，最刻右双…

    萧炎有些姐体，萧炎曾所若是自己受懒暗了决匠：体一族人，级悄无想中，也能资三问笑受件，及会高全中，不会有一次人儿家族外级在庭魔个高听，平于鼻由，这是才同裂…到师最阵员花股样长他笑钱没了感备来，他不“族长，这几水旧是直想性好奇的西给倒注解什，到萧炎记怎么挑战了，所让得真如玄不”对不过比花就早核拿起千先被脉六忍不小小的竟方随所说，而差门父，最后炎儿狠的，若且是家其下被能被高去，高这是摸绍还只有生透大现下隐情根，越是姐秘名着后爱的今天

    明听的酷势，萧炎傻子，了老赋随意力的时重了一句何才淡缓与模礼，竟然众也实在最高种会强书丽，：什望向这闲的处辰，制要没材畏，新身，本种能性上升，方由将到络所果，什么二步脚用，不是体内震从，所她算是亮了极衣类养之点，这种算是好最先这配谓了口子，平及了吧？”

Epoch 1/1
312/312 [==============================] - 150s 480ms/step - loss: 2.7265

----- Generating text after Epoch: 8 -----
----- diversity: ----- 0.2
----- Generating with seed: "已经亲自下山，替嫣然准备一方药材中地最后" -----

已经亲自下山，替嫣然准备一方药材中地最后十年一种，都是有些无奈的叹了一口气，一些小脸上的人有些无奈，萧炎的身材材必，让得萧炎心头有些无奈的叹了一口气，萧炎的话，却是一些长长的叹了一口气，一道。

    “你的吧？”一步，不过，萧炎的身后，却是一些长长的话，那么是为人的身后，还有一些无奈的萧炎，萧炎有些无奈的叹了一口气，一道道：“不知道。”

    “你……

    “你的吧？”萧炎小脸庞上的话，萧炎的话，却是一些长长老者说了笑声，
    在萧炎的身后，萧炎的身后，萧炎的身后，却是一些长长的话？

    在一脸淡淡的话语一个小小的话者一起，薰儿小脸上了笑容，萧炎的身形，却是一些长长老者有些强者的笑容，实而一个人，都是有些无奈的叹了一口气，一些小脸上的人，也是一些一名的年：“不知道。”萧炎小脸上的笑容，而且还有些无奈的叹


脑海中缓缓的回忆着以往的一些旧事，萧炎嘴角角八抽半惊之后，凉气翻掌断动之。

    两 斗中声中级声，薰儿上面平百淡大的黑位崖狂微默的眼级，不过五望着：“父亲再次少爷，十认脸人吧之目是不神强化出溢头，就算神带着了摇头。

    括出戏造看的少年，眼落交那轻正正在经人便也会性感到，和萧漂从却是有些急不吧。

    “三少爷将三次赋因的我想要家任身奇过也没有眉家细意的想要不理什么牙的为以，”究从大沉而走，在功温成大最斗方的章些这隐取抬是东西，有些习名的怪级魔低的追摸，萧炎微微一足，嘴角猛的闪微一终打过身，萧炎再次少女，虽然能重她一究坐，姐斗之气大黑用之下，很难。

    “若是这会能够让你知低也没事散之心的，就是少年女女天内，无备想想还没起挺去其他听不知让他先几很什么人都是嫣然过领。

    在萧炎隐第的如冲魔狠的闷的老清着浅体有些无米的在关人中之下。

    少年对着清淡的那条身体上坐了下头，萧

Epoch 1/1
312/312 [==============================] - 124s 397ms/step - loss: 2.3552

----- Generating text after Epoch: 12 -----
----- diversity: ----- 0.2
----- Generating with seed: "浮现一抹狰狞。再不迟疑，手中的大刀，狠狠" -----

浮现一抹狰狞。再不迟疑，手中的大刀，狠狠的在萧炎的水液，不过萧炎的身旁，萧炎的笑道。

    “这小，，我是一个斗之气大陆之上，我也是不会。”

    “你就在了一天，萧炎不过这才的声音，却是有些无奈的道。

    “这小，，我是一个月时，我能够我这老的，不过萧炎，你也是可能…”

    “你的，我是一个小小，，萧炎心头有些无奈的道：“这不是萧炎哥哥，不过这才点了点头，萧炎的笑道。

    “这小，，我是一个月时，我能让我这是不会他的个能有些感到这种种而想要成为斗气了，你的修炼天赋，也不可能把你难道，不过，你还是以及你的修炼，这是一个小小，萧炎也是不过却的被萧炎出现了他的一位中，微微一声，转身子在双眼中的水。

    “这小，，我是一个能有点点，你们的是族人，我们，可有些强年的斗技，而是一个人，不过，你就是萧炎哥哥不过不会出现，



    “破！”小脸肃然的一声冷喝，红“纳兰草晓程化道：情“转端叫一底手吧…水无什难材料，这位而踏还有强不知命功的萧战，所得有他们萧炎带着心头，话你中实在与魔使为人三年高旋震惊看斗外想被称首吟老的火掩，萧炎皱起顿住，目光转中那一个清雅的小小小嘴色第位。

    身热黑难火掩了啊遍润的问句，萧炎苦笑一主，爱动有了额头。

    闻言，萧炎闭上发觉，哥手们萧战的两声段只了第待者冲灵的笑容，那才清雅的种白抹处种水眼黯高探轴吞，丹药想了即造先溢压的与此次而材相，也！同住不是为与烈师三位斗气强，可有势“。”

    还？功师萧炎等着这平师不是一受族长看早不要满心注声。

    只非为笑成之下，”萧的不微少要…色药老吐了一口转，眼色下手且那白气水寻手老烈的刻之心，更待为皱巴他来他带什一青人红吐了，小脸笑容：…”名且当开她的人流中瞟长急叹了一个追而处，虽然下该丝收一名发了而什么从位长老将察金，惊晓的大多少年，望着萧炎

Epoch 1/1
312/312 [==============================] - 118s 377ms/step - loss: 2.0733

----- Generating text after Epoch: 16 -----
----- diversity: ----- 0.2
----- Generating with seed: "

    “炎儿，没事吧？刚才萧玉" -----



    “炎儿，没事吧？刚才萧玉就是一个有些个月的，现在的这才回来，不过，现在的他，也是被难道小家族的速度，不过，我们也不再有什么有些个有点点，一点，在这一个小时，同时，这一个小小，却是这个月时候的话，他也是一名有一个小姐，可以这可能，也只是一个人有点发生的小脸，却是这些不断的乎来到了一个不第七段，来到萧炎一年，恐怕也不是什么由于一个，萧炎心头有些发麻的前影，萧炎心头有些发冷的道：“我知道了，我要如何不要是你这点头，我是他这一个自己么？”

    “萧炎哥哥。”

    “呵呵，我要不要，你也不是这所说，我要是你的吧，？”

    “呵呵，我要不要，我你以后，你也是有人，你的！”这是生五，可你还是萧炎哥哥有点无点点点点头上现了一个，然后一个小时，就是萧炎的说过了一个月的，有些年人，在这个之中，最后的天赋，不是一个小时，这小家伙

...


着咄咄逼人的加列奥，周围一些平日与萧炎关乎得秀交话过打在身旁的那位微虽与怒气的热赋的族人，却是特人的时候，家族之任满只能来第七段，主高只颤转乱身直少需把找牙门口无些苦老的道：炎带着下来，以前的那些震撼，年力的此药，眸子有些就不耐的为那惊足，真的打格在带起。

    “萧炎哥哥，我并不长寻现在就被加列毕精缩的只是不知些以后，萧炎这惊这话与脸心中，足有着青少时，以老属下！”摸身后，两着萧炎反难的老，随手瞥手眼眸天

    看来对儿少年的对着对下乌坦城让先生小小的加列缓缓少女冷笑道：“半没有纳兰萧叔配合着你几分天赋，就算在自己家族之摸。”

    然后在全黑清脆的破之中，将萧炎看似目见二长老以强的反推。

    黑石碑之上，望着势的加列毕男着望着面，三少行出的紫追制，可修炼你强低头的！”叹音小手中，萧炎望着脚没自己在从上那月后的知道之外去，只能也能成为斗气的以天赋，打算？加列毕赶脚上的震撼了。

    此

----- diversity: ----- 1.2
----- Generating with seed: "着咄咄逼人的加列奥，周围一些平日与萧炎关" -----

着咄咄逼人的加列奥，周围一些平日与萧炎关低影，的提醒能声让刚双水妃炼着液行紧味时出少女脸庞，有些惊复的修炼一阵冷师，一脚脸似低有些停，有些身动与哥击，最后直挺带出来。

    “以萧。”

    听着萧媚的脸色随足冲着如最重恍的立冲缓而因为回天，清微灵光完起前来，切步的“含

    望着那击自己的寂静后肩，晶后，薰儿忽然一转，眨道：“薰儿便灵，这是经个大弧分”，个位身影

    “似乎？”眼到中轻的第天，萧炎冷静道：“两这挑掌相波的东西，？”萧炎转过那黑微微点头，萧炎心中。

    “究里，整你！”向儿中，萧炎心见断在笑着少女的皇蛇一待之要，手臂剑以前的狠少，谓于成：并不特修炼天才道：“我十八波手，平能不天那用重族的等射，萧而伸的停下炎白了白会远场稳场外引的药牢山炼制不对会，萧以受如此为水，这样间闷！”

    转身并不角顺着现后，气堂放下！”

    最后犹如被一少见，这才风笑：“这待

Epoch 1/1
312/312 [==============================] - 120s 386ms/step - loss: 1.0764

----- Gen


指微微一颤，药老摇摇晃晃的飘了出来，望着脑袋的萧萧炎觉，虽然他知晓这萧与走过当他得意外，不知为自她的会成，如果百雅少女还是小脸上，深吸了一口气，毫不无头，从那具备到了一声来，已经被萧媚掌心之动着对手，高台上那一道：“我知哥外染感嘴不纠？”萧炎双手心。，萧炎那股叹，一果有些在萧炎的体内，便是一眼在进入中，沉丽的月中，然后将无族中，却是退步行过物。

    忽然猛的火下的于训，回先了回去修何人。

    盘充对自己心中平淡的金光萧炎无气的对着萧家族瞳缓缓的片美缝萧炎，淡淡的道：带着一脸淡然的亲容，就是他们美丽的少女并未闻略微小手，已眨狠所的月出那反中的子下近淡如咬牙日后，将成子木盆之中，身清平发的争进回修同后，一名实力有地让得萧宁脸乎手难一抹挑了，所以，可没自己这炼药常乎话，充多人对不眼心所之处，惊最的极为神秘的道：““现在我…”种目光已扫向萧炎轻叹了一口气，，在年中的落有

    脸色冷静的萧炎的萧玉，旋即丝差“

----- diversity: ----- 1.2
----- Generating with seed: "指微微一颤，药老摇摇晃晃的飘了出来，望着" -----

指微微一颤，药老摇摇晃晃的飘了出来，望着平说无能寻萧炎可旁的得模人在身上耸了噶日后，击得应畅成不得，此次并不是一直是曲高，纤强冷的点了点之之上，一声向惊见的行人。

    望着传来这青春少年清秀有的脸速瞧着灵丽的事，难笑道打闷身深平父很大名头，嗅样么美眸子的淡淡的那低年忽场紧五，轻一落坚样，萧炎有些此树的要话着反前…

十给一。

    极自旁的淡异，萧炎那身想红衣以及完美的这东红润着萧炎应说，那对方这袍个定了大眼的金化，略微急包的好起，老有人求的合格。

    望着寂静手，目光淡扫，只是一天了笑道：“毕你反强，炎儿子出会。”

    耸了摊

    望着旁的察觉斗之气已经让你虽然前这此异对却是这废物。

    嘴使萧战一直，^_打宁缓缓眉头一嘴，其他，而不着痕迹的脸嘴上，前未牙看视着家族的干日的话需要求石回炎去师，是起得书神”

    ……

    “八了七！”轻强了未来衬道

Epoch 1/1
312/312 [==============================] - 118s 378ms/step - loss: 0.9781

----- Gen


，难道…是飞行斗技？”

    “只是在再次他央你能知晓这年之后，不管我力不用了过来，萧家紫注心中他没有你…试什么家教美说，需要萧炎曾经是是深意，得过势强叹了一必的话，他第今天这不天比修炼萧萧震加命的越来越近了一升的黑衫少年。

    布入一斗技小小的盆之上，却未放你都不得被眼下心技保不更更是“风多年水性，在会她会内，门前没有行家族都势摸注哥，不多光本力放带了一道做了这女人过了还不世旁的浅月的。

    都头不喃了。

    萧炎的一笑，薰儿轻声，混了落射出的拉了下来，散掌与清意，显然，一些她斜靠之现，地上为斗人找旧尊心中去前一一些年前人前向痕迹的犹如看炎这请真是女人傻子。

    对着这天臂怨性的小，虽然同样是在心中的最越却是俏名戏谑的微微点，极为战脸，却是所在挑了过先他所是屑，虽然在身材教近无吸，萧炎的道：请家族中能够眼，不住美萧宁双双瞳中，是一触射出与几长长下：“还有九段！你到口是是求些云高级

----- diversity: ----- 1.2
----- Generating with seed: "，难道…是飞行斗技？”

    “" -----

，难道…是飞行斗技？”

    “并不少需需要看来不远族…吧？”等中众是略微震撼，这声中斥在心才大同收被，所有的摸了许多。

    修炼与攻势力的此时都纤各奈的阵练，急忙行着前试美丽，传苦娇仪云狰六，实力前能再无阵的话语气，长老脑双妒房的家族高也是那着脸蛋的冷够，不露散出行出历右…

    被萧炎的萧炎，有些玉些咳，虽然好在拍修炼缓缓有些家族之上，而红融之声，却是远将会尽讪知道。

    溢下这家伙的兴下，眼光铁瞟着萧炎躲寒，在猛股出寒去了玄阶斗技：吸师，是用足够张一用之天，而现在心头有些终内行去。

    学着攀台上的玉手，玉齿的一把红润的脸颊，势力的无非腾还所得：萧炎左右，还得怨东西，见到萧玉三人目光双他空实的萧炎，薰儿竟然如“回到他挑战斗，不过每老却没有小吸，我否如重所不强，再不连这八大的出声询问道。

    干出以来萧薰儿亲水交

    说过是小来的一教拍二长老一身反炎，等你而与

Epoch 1/1
312/312 [==============================] - 124s 397ms/step - loss: 0.9075

----- Gener


的道：“那家伙就是个怪胎，怎么可能用常理，你很也就不会了？”萧炎皱了虽然萧炎微急白了半外咬牙高手，欲时着宁，自己家伙鹰什么级转的头，待会什么落黑晶化出可不过了。

    “没亲自然不天，自我这是个斗技的差有位喝出现在玉情的差距道，虽然反而媚，翻身传与未左转，一些人都是被形事证的修长。

    （新狞等青，现在就才看距已经赶沉尊贵炼样的信！”

    萧炎的，这女子，在静坐没认为斗气，究连那方她东他小牙哦。”萧战小脸笑，微笑道：“萧媚！

    “这是事！”你那战几年人斗者之下，其重物的修炼身子，地上方让得起静不屑与看拥，萧炎目微优“拉向族长老来之后，淡淡的气子木略微安静了下来，不过他的他，脸庞上的不地面低盈，不过轻美自美眸地这双利之上。

    然后方气只在萧战有些只有，当然人了不知为家族长，想起了一时：“红现老人！”萧炎心中犹如同话是萧炎一句冷笑，摇了摇头，心头一句大击，笑着嘴中的淡天的弧地，便是

----- diversity: ----- 1.2
----- Generating with seed: "的道：“那家伙就是个怪胎，怎么可能用常理" -----

的道：“那家伙就是个怪胎，怎么可能用常理人爷和你你是说得他嫣然于衣必长是忽然有多。”

    俏和身姿虽转偏服着老者笑道。”

    脸庞上的小踏未萧炎无姐，冷下睛不了笑，不角忽然一口气表意。

    却是稚嫩的风手，只经变自冷无修长的一触，萧炎要因为一枚斗气大陆，不过为强于收下，从一般的也不真缓缓行地，刚好。

    从不如何能乱，至于不九挺吧…千打得八段斗之别人高信上手，兴视入失大…把而反个，宁要消失…

    “成为这爷来摇冷青灿？”在铁被！”苦头，狠的消天的踏步身上，成手青凤坠质的一边，请耐萧媚心前的火属性家。”

    萧战对着萧炎微微的说，虽然对他有些额视，多知道，父亲样时候强小的族大学人。

    微微流几分而成

    所以，以一仪东西，以和就是“挑战的…”

    “一阵兴奋年实是萧炎模样，还以睛全放方！年动人，都几掩背势在够亮身姿兴话之中。

    

Epoch 1/1
312/312 [==============================] - 124s 398ms/step - loss: 0.8438

----- Generatin


，希望那些家伙不会太过分吧…”心中轻叹了一口气，废余在坐在狰处身体的内隐之人。

    经在一个人有着然后推门进行着对，还要不是没有两声都来抹清秀，对不是由细什么微微强者下脸平静的前人还的萧炎从那俏脸缓缓随意的笑了出了，就是被我微笑道。

    更是回来了不过，而当谓向洒远后高等级的，可以十可体，想必炼进去属于这么求理出了，家伙也是察觉到一名真强，你是他现在炼？了资格，在谁你了这斗之上的实力，不过是在一任多少年，是他能把这类被要成第三段斗之气？如种两人加到薰儿白了易要不说，她也难道，让得萧炎不过虽然的修炼，熟是那天才少年微微的笑着下来，表情出来的呼，而失完全力：身材亲喝人，以她一个老，而当时中的高木，才是呵呵一一股美丽拍美感打，以及那淡淡的才笑了一声，随意有着淡淡的千斤事，到得并未白切几句小手票票票！起一下。

    一阵如果你对他的注目，自己心情让得他随意，为让你随便究竟有姿狠！”

    被必名之后的

----- diversity: ----- 1.2
----- Generating with seed: "，希望那些家伙不会太过分吧…”心中轻叹了" -----

，希望那些家伙不会太过分吧…”心中轻叹了笑道。

    片刻，萧炎已难轻，炎了眨，然后，天后，仅仅是呵要来。

    “我儿哥小打下那么过理年，身几急分淡…大等将身子，嘴到进何时许能显不动，等着成年仪式了容，看见俏应神天的寻会就算强东天的大厅。

    三少女失只会大容身意，懒懒的也是缓缓的缓行至刚口，一名年龄的那内真段了…”炎只得向头点了少下，少年轻轻的归急缓缓缓缓坐然，望着淡淡的亮光。

    美眸中盯着不意几分着锐意完紫低低的门过那黑时候握不意的薰儿，微微点头，小手，当气在带着引到了大出，正的精技…

    一大步进过，萧炎与萧浑清秀自体的日锤，竟然是起族在从一意，人。

    （新失等十还难以，他如此八色斗技的别人打成了一障好，迷人人中这种女人，所是重气，没有强极句句想干了吼话，虽然在同时忽然也不住的追？如果你以现在真种不友，尔势在这超日的身股走来。

    三年之后“水性，经后



In [23]:
# import pickle

# with open('model_008099_0821.pkl', 'wb')as f:
#     pickle.dump(model, f)